__Общая задача:__ создать скрипт для формирования витрины на основе логов web-сайта.

In [1]:
import pandas as pd        # импортируем необходимые библиотеки
import numpy as np
import psycopg2
from datetime import datetime
import re
import httpagentparser

In [2]:
# все что нужно для спарка
from pyspark.sql import SparkSession
from pyspark.sql import Window
from pyspark.sql.functions import *
import pyspark.sql.functions as F
from pyspark.sql.functions import regexp_extract 
from pyspark.sql.functions import lit

In [70]:
# создаем спарк-сессию
spark = SparkSession.builder \
  .master("local[1]") \
  .appName("SparkFirst") \
  .appName("Timeout Troubleshooting") \
  .config("spark.executor.memory", "10g")\
  .config("spark.executor.cores", 5) \
  .config("spark.dynamicAllocation.enabled", "true") \
  .config("spark.dynamicAllocation.maxExecutors", 5) \
  .config("spark.network.timeout", "600s") \
  .config("spark.shuffle.service.enabled", "true") \
  .getOrCreate()

In [4]:
base_df = spark.read.text('access.log')
base_df.printSchema()

root
 |-- value: string (nullable = true)



In [5]:
base_df.show(3, truncate=False)


+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                                                                                                                                                                                                     |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|54.36.149.41 - - [22/Jan/2019:03:56:14 +0330] "G

In [6]:
(base_df.filter(base_df['value'].isNull()).count()) # проверим есть ли нулевые строки в исходном датафрейме

0

In [7]:
sample_logs = [item['value'] for item in base_df.take(5)]  # выберем небольшую выборку из основного датафрейма для проверки регулярок
sample_logs

['54.36.149.41 - - [22/Jan/2019:03:56:14 +0330] "GET /filter/27|13%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C%DA%A9%D8%B3%D9%84,27|%DA%A9%D9%85%D8%AA%D8%B1%20%D8%A7%D8%B2%205%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C%DA%A9%D8%B3%D9%84,p53 HTTP/1.1" 200 30577 "-" "Mozilla/5.0 (compatible; AhrefsBot/6.1; +http://ahrefs.com/robot/)" "-"',
 '31.56.96.51 - - [22/Jan/2019:03:56:16 +0330] "GET /image/60844/productModel/200x200 HTTP/1.1" 200 5667 "https://www.zanbil.ir/m/filter/b113" "Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build/HuaweiALE-L21) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.158 Mobile Safari/537.36" "-"',
 '31.56.96.51 - - [22/Jan/2019:03:56:16 +0330] "GET /image/61474/productModel/200x200 HTTP/1.1" 200 5379 "https://www.zanbil.ir/m/filter/b113" "Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build/HuaweiALE-L21) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.158 Mobile Safari/537.36" "-"',
 '40.77.167.129 - - [22/Jan/2019:03:56:17 +0330] "GET /image/14925/productModel/100x100 HTTP/

При помощи регулярных выражений спарсим необходимые данные. Из лога я взяла не все данные, только хост, метод, протокол, статус и самое главное строку с User agent-информацией.

In [8]:
host_pattern = r'(^\S+\.[\S+\.]+\S+)\s'              #регулярное выражение для хоста
hosts = [re.search(host_pattern, item).group(1)
           if re.search(host_pattern, item)
           else 'no match'
           for item in sample_logs]
hosts

['54.36.149.41', '31.56.96.51', '31.56.96.51', '40.77.167.129', '91.99.72.15']

In [9]:
method_uri_protocol_pattern = r'\"(\S+) (.*?) (\S+)\"'     #регулярное выражение для метода
method_uri_protocol = [re.search(method_uri_protocol_pattern, item).groups()
               if re.search(method_uri_protocol_pattern, item)
               else 'no match'
              for item in sample_logs]
method_uri_protocol

[('GET',
  '/filter/27|13%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C%DA%A9%D8%B3%D9%84,27|%DA%A9%D9%85%D8%AA%D8%B1%20%D8%A7%D8%B2%205%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C%DA%A9%D8%B3%D9%84,p53',
  'HTTP/1.1'),
 ('GET', '/image/60844/productModel/200x200', 'HTTP/1.1'),
 ('GET', '/image/61474/productModel/200x200', 'HTTP/1.1'),
 ('GET', '/image/14925/productModel/100x100', 'HTTP/1.1'),
 ('GET',
  '/product/31893/62100/%D8%B3%D8%B4%D9%88%D8%A7%D8%B1-%D8%AE%D8%A7%D9%86%DA%AF%DB%8C-%D9%BE%D8%B1%D9%86%D8%B3%D9%84%DB%8C-%D9%85%D8%AF%D9%84-PR257AT',
  'HTTP/1.1')]

In [10]:
status_pattern = r'\s(\d{3})\s'                     #регулярное выражение для ответа
status = [re.search(status_pattern, item).group(1) for item in sample_logs]
print(status)

['200', '200', '200', '200', '200']


In [11]:
user_agent_top=r'\S+\S+\".\"([^\(]+)\((.*)\)'      #регулярное выражение для начала UA-строки
user_agent = [re.search(user_agent_top, item).group(1) for item in sample_logs]
user_agent                              #мы его использовать в итоговой таблице не будем так, как оно для всех одинаковое и не несет полезной информации

['Mozilla/5.0 ',
 'Mozilla/5.0 ',
 'Mozilla/5.0 ',
 'Mozilla/5.0 ',
 'Mozilla/5.0 ']

In [12]:
user_agent_end=r'\((.*) '                       #регулярнео выражение для окончания UA-строки
user_agent = [re.search(user_agent_end, item).group(1) for item in sample_logs]
user_agent

['compatible; AhrefsBot/6.1; +http://ahrefs.com/robot/)"',
 'Linux; Android 6.0; ALE-L21 Build/HuaweiALE-L21) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.158 Mobile Safari/537.36"',
 'Linux; Android 6.0; ALE-L21 Build/HuaweiALE-L21) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.158 Mobile Safari/537.36"',
 'compatible; bingbot/2.0; +http://www.bing.com/bingbot.htm)"',
 'Windows NT 6.2; Win64; x64; rv:16.0)Gecko/16.0 Firefox/16.0"']

In [25]:
logs_df = base_df.select(regexp_extract('value', host_pattern, 1).alias('host'),           # теперь объединяем все вместе
                         regexp_extract('value', method_uri_protocol_pattern, 1).alias('method'),
                         regexp_extract('value', status_pattern, 1).cast('integer').alias('status'),
                         regexp_extract('value', user_agent_end, 1).alias('user_agent_end'))
logs_df.show(10, truncate=True)
print((logs_df.count(), len(logs_df.columns)))

+-------------+------+------+--------------------+
|         host|method|status|      user_agent_end|
+-------------+------+------+--------------------+
| 54.36.149.41|   GET|   200|compatible; Ahref...|
|  31.56.96.51|   GET|   200|Linux; Android 6....|
|  31.56.96.51|   GET|   200|Linux; Android 6....|
|40.77.167.129|   GET|   200|compatible; bingb...|
|  91.99.72.15|   GET|   200|Windows NT 6.2; W...|
|40.77.167.129|   GET|   200|compatible; bingb...|
|40.77.167.129|   GET|   200|compatible; bingb...|
|40.77.167.129|   GET|   200|compatible; bingb...|
|66.249.66.194|   GET|   200|compatible; Googl...|
|40.77.167.129|   GET|   200|compatible; bingb...|
+-------------+------+------+--------------------+
only showing top 10 rows

(10365152, 4)


Проверим схему нашего датафрейма.

In [14]:
logs_df.printSchema()

root
 |-- host: string (nullable = true)
 |-- method: string (nullable = true)
 |-- status: integer (nullable = true)
 |-- user_agent_end: string (nullable = true)



In [16]:
bad_rows_df = logs_df.filter(logs_df['host'].isNull()| 
                             logs_df['method'].isNull() |
                             logs_df['status'].isNull() |
                             logs_df['user_agent_end'].isNull())                    
bad_rows_df.count()   #пустых строчек нет, это хорошо!

0

Приступим к проведению анализа и построению витрины:
- Суррогатный ключ устройства

- Название устройства

- Количество пользователей

- Доля пользователей данного устройства от общего числа пользователей

- Количество совершенных действий для данного устройства

- Доля совершенных действий с данного устройства относительно других устройств

- Список из 5 самых популярных браузеров, используемых на данном устройстве различными пользователями, с указанием доли использования для данного браузера относительно остальных браузеров 

- Количество ответов сервера, отличных от 200 на данном устройстве

- Для каждого из ответов сервера, отличных от 200, сформировать поле, в котором будет содержаться количество ответов данного типа

Для определения устройства и браузера воспользуемся готовым парсером (https://pypi.org/project/httpagentparser/#description) (импортировали его вначале):

Метод httpagentparser.detect(s) дает ответ вида (пример) {'platform': {'name': 'Android', 'version': '6.0'}, 'os': {'name': 'Linux'}, 'bot': False, 'dist': {'name': 'Android', 'version': '6.0'}, 'browser': {'name': 'Chrome', 'version': '66.0.3359.158'}}

In [52]:
#тестовая строка
s='Linux; Android 6.0; ALE-L21 Build/HuaweiALE-L21) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.158 Mobile Safari/537.36'
y=httpagentparser.detect(s)
print('Платформа',y['platform']['name'],',браузер', y['browser']['name'])


Платформа Android ,браузер Chrome


In [156]:
#@F.udf                 # создадим пользовательскую функцию на основе парсера, которая возвращает название платформы
def device_def(x):
    y= httpagentparser.detect(x)
    if 'platform' in y:
        return y['platform']['name'] 
    else:
        return 'None'      # ниже протестируем работу функции

In [142]:
device_def('Linux; Android 6.0; ALE-L21 Build/HuaweiALE-L21) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.158 Mobile Safari/537.36')

'Android'

In [157]:
device_def('compatible; AhrefsBot/6.1; +http://ahrefs.com/robot/')

In [158]:
logs_platform = logs_df.withColumn('platform', lit(device_def(col('user_agent_end'))))\
                .show(3)                                  # создадим новый столбец с платформой

+------------+------+------+--------------------+--------+
|        host|method|status|      user_agent_end|platform|
+------------+------+------+--------------------+--------+
|54.36.149.41|   GET|   200|compatible; Ahref...|    null|
| 31.56.96.51|   GET|   200|Linux; Android 6....|    null|
| 31.56.96.51|   GET|   200|Linux; Android 6....|    null|
+------------+------+------+--------------------+--------+
only showing top 3 rows



In [131]:
@F.udf                 # создадим пользовательскую функцию на основе парсера, которая возвращает название браузера
def browser_def(x:str)->str:
    y= httpagentparser.detect(x)
    if 'browser' in y:
        return f"{y['browser']['name']}" 
    else:
        return 'None'                   # ниже протестируем работу функции

In [119]:
browser_def('Linux; Android 6.0; ALE-L21 Build/HuaweiALE-L21) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.158 Mobile Safari/537.36')

'Chrome'

In [132]:
logs_browser=logs_df.withColumn("browser", lit(browser_def(F.col("user_agent_end"))))\
         .show(3)                                  # применим функцию и создадим новый столбец с браузером 

Py4JJavaError: An error occurred while calling o553.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 81.0 failed 1 times, most recent failure: Lost task 0.0 in stage 81.0 (TID 427) (host.docker.internal executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:192)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:166)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec.evaluate(BatchEvalPythonExec.scala:82)
	at org.apache.spark.sql.execution.python.EvalPythonExec.$anonfun$doExecute$2(EvalPythonExec.scala:131)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitions$2(RDD.scala:853)
	at org.apache.spark.rdd.RDD.$anonfu

Кэшируем logs_df для дальнейшего анализа:

In [ ]:
logs_df.cache()

Количество пользователей - это количество уникальных хостов:

In [41]:
total_cust = logs_df.select(count_distinct("host")).show()

+--------------------+
|count(DISTINCT host)|
+--------------------+
|              258606|
+--------------------+



Количество пользователей устройства - это пользователи, сгруппированные по устройству:

In [ ]:
#device_users=logs_df.select('host').groupby('platform').count().sort('count', ascending=False).show()

Количество совершенных действий для данного устройства - это группировка по колонке и подсчет методов (GET, POST):

In [ ]:
#device_actions=logs_df.select('method').groupby('platform').count().show()

Всего действий - это подсчет значений колонки методов:

In [44]:
total_actions = logs_df.select('method').count()
total_actions

10365152

Доля совершенных действий с данного устройства относительно других устройств - это 

Список из 5 самых популярных браузеров, используемых на данном устройстве различными пользователями, с указанием доли использования для данного браузера относительно остальных браузеров - это 

In [ ]:
top_5_b= logs_df.select('browser').groupby('platform').count().sort("count", ascending=False).limit(5).show()

In [47]:
total_status=logs_df.groupBy('status').count().sort('count', ascending=False).show() 

+------+-------+
|status|  count|
+------+-------+
|   200|9579825|
|   304| 340228|
|   302| 199835|
|   404| 105011|
|   301|  67552|
|   499|  50852|
|   500|  14266|
|   403|   5634|
|   502|    798|
|   400|    376|
|   401|    323|
|   777|    211|
|   408|    112|
|   504|    103|
|   414|     17|
|   405|      6|
|   206|      3|
+------+-------+



Количество ответов сервера, отличных от 200 на данном устройстве:

In [83]:
answers_ne200 = logs_df.filter(logs_df.status != '200').groupby('platform').count()
answers_ne200.show()

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `platform` cannot be resolved. Did you mean one of the following? [`method`, `status`, `host`, `user_agent_end`].;
'Aggregate ['platform], ['platform, count(1) AS count#716L]
+- Filter NOT (status#155 = cast(200 as int))
   +- Project [regexp_extract(value#0, (^\S+\.[\S+\.]+\S+)\s, 1) AS host#153, regexp_extract(value#0, \"(\S+) (.*?) (\S+)\", 1) AS method#154, cast(regexp_extract(value#0, \s(\d{3})\s, 1) as int) AS status#155, regexp_extract(value#0, \((.*) , 1) AS user_agent_end#156]
      +- Relation [value#0] text


Для каждого из ответов сервера, отличных от 200, сформировать поле, в котором будет содержаться количество ответов данного типа:

In [ ]:
db_con = psycopg2.connect(database='testdb',     # создадим подключение к созданной базе данных exrate
                        user='postgres',
                        password='pass',
                        host='localhost',
                        port=5435)
cur = db_con.cursor()

In [ ]:
cur.execute(""" CREATE TABLE btc_Jan2023(
    id SERIAL PRIMARY KEY,
    date DATE,
    val_id VARCHAR,
    base_rate DECIMAL,
    val_base VARCHAR
)
""") 
db_con.commit()        

In [ ]:
!pip install httpagentparser

In [ ]:
import httpagentparser


In [ ]:
s = "Mozilla/5.0 (X11; U; Linux i686; en-US) AppleWebKit/532.9 (KHTML, like Gecko) \
        Chrome/5.0.307.11 Safari/532.9"

In [ ]:
print(httpagentparser.simple_detect(s))

In [ ]:
cur.close()
db_con.close()